In [1]:
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import random

In [2]:
# Lấy về danh sách các proxy IP và port từ https://sslproxies.org
def get_proxies():
    url = 'https://sslproxies.org/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    proxies = []
    count = 0
    for row in soup.find_all('tr')[1:]:
        # Chỉ lấy khoảng 20 proxies
        if(count>=20):
            break
        columns = row.find_all('td')
        # phải có ít nhất hai elements thì mới lấy
        if columns[4].text == "elite proxy" and columns[5].text == "yes":
            ip = columns[0].text
            port = columns[1].text
            proxies.append(f'{ip}:{port}')
        count +=1
    return proxies

def get_vietnamese_proxies():
    url = 'http://free-proxy.cz/en/proxylist/country/VN/all/ping/all'
    driver = webdriver.Chrome()
    driver.get(url)
    proxies = []
    proxy_table = driver.find_element(By.ID, 'proxy_list')
    count = 0
    # Tìm table có id 'proxy_list'
    rows = proxy_table.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        if(count>=10): break
        # Find all cells (columns) in the current row
        cells = row.find_elements(By.TAG_NAME, 'td')
        # Lấy ra content của cột đầu, cột hai (index 0 and 1)
        if len(cells) >= 2:
            first_column_value = cells[0].text.strip()
            second_column_value = cells[1].text.strip()
            fifth_column_value = cells[6].text.strip()
            if(fifth_column_value == "High anonymity"):
                proxies.append(f'{first_column_value}:{second_column_value}')
                print(f"First Column: {first_column_value}, Second Column: {second_column_value}")
                count+=1
    # Đóng WebDriver
    driver.quit()
    return proxies

# lấy ra danh sách các proxies
# proxy_list = get_proxies()

# lấy ra danh sách các proxies của Việt Nam
vn_proxy_list = get_vietnamese_proxies()

# Chọn một proxy ngẫu nhiên
# random_proxy = random.choice(proxy_list)
vn_random_proxy = random.choice(vn_proxy_list)
# print('Proxy: ' + random_proxy)
print('VN Proxy: ' + vn_random_proxy)

First Column: 171.247.241.38, Second Column: 1080
First Column: 171.254.216.146, Second Column: 1080
First Column: 171.247.240.26, Second Column: 1080
First Column: 171.248.208.57, Second Column: 1080
First Column: 171.248.212.157, Second Column: 1080
First Column: 171.247.245.88, Second Column: 1080
First Column: 171.247.242.159, Second Column: 1080
First Column: 171.247.242.244, Second Column: 1080
First Column: 171.248.222.9, Second Column: 1080
First Column: 171.248.214.175, Second Column: 1080
VN Proxy: 171.248.208.57:1080


In [3]:
def get_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument(f'--proxy-server={vn_random_proxy}')
    driver = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
    return driver

# AUTO VIEW VLOGNOVEL

In [7]:

# Hàm auto scroll, refresh sau nhiều lần và click next chương bằng Selenium
def open_and_refresh_websiteVL(driver,url, interval_seconds, number_of_chapter = 0):
    try:
        driver.get(url)
    except ConnectionRefusedError:
        new_driver = get_chrome_driver()
        open_and_refresh_websiteVL(new_driver, url, interval_seconds, number_of_chapter)
    except WebDriverException:
        driver.quit()
        return
    count = 0
    while(count<=1):
        for _ in range(5):
            # Dùng JavaScript để từ từ cuộn xuống
            driver.execute_script("window.scrollBy(0, 220);")
            time.sleep(1)
            driver.execute_script("window.scrollBy(0, 220);")
            driver.execute_script("window.scrollBy(0, 220);")
        driver.refresh()
        time.sleep(interval_seconds)
        count+=1
    if(number_of_chapter > 60):
        driver.quit()
    if(driver.find_elements(By.XPATH, '//*[@id="chapter-article"]/section/div[6]/a[5]')!=[]): 
        buttonNext = driver.find_element(By.XPATH, '//*[@id="chapter-article"]/section/div[6]/a[5]').click()
    nexturl = driver.current_url
    open_and_refresh_websiteVL(driver,nexturl, interval_seconds, number_of_chapter+1)

# Set the website URL
website_url = "https://vlognovel2.com/than-kiem-luu-lac-phap-gioi/chuong-2-alex-6581996a27111c9a2b05b38c.html"
website_url_VL33 = "https://vlognovel2.com/than-kiem-luu-lac-phap-gioi/chuong-33-tat-xau-658ac491e98dbe584d01407e.html?next=chapter"

# Số giây cách
refresh_interval = 3

d = get_chrome_driver()
# Gọi đến function auto refresh và auto click next chương
open_and_refresh_websiteVL(d,website_url, refresh_interval)

MaxRetryError: HTTPConnectionPool(host='localhost', port=58984): Max retries exceeded with url: /session/cda28a7f948610d6749604cc59e2d3a7/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002547788E050>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))